# HyperCLOVAX Fine-tuning for Korean Text De-obfuscation

이 노트북은 Naver HyperCLOVAX-SEED-Text-Instruct-0.5B 모델을 한국어 문자열 비난독화를 위해 fine-tuning하는 과정을 보여줍니다.

## 목표
- 난독화된 한국어 텍스트를 원본 텍스트로 복원하는 모델 훈련
- LoRA (Low-Rank Adaptation)를 사용한 효율적인 fine-tuning
- 다양한 텍스트 유형 (구어체, 뉴스, 문화, 전문분야, 조례, 지자체웹사이트)에 대한 성능 평가

## 1. 환경 설정 및 라이브러리 설치

In [ ]:
# GPU 확인
!nvidia-smi

# 필요한 패키지 설치
!pip install -q transformers
!pip install -q peft
!pip install -q trl
!pip install -q datasets
!pip install -q bitsandbytes
!pip install -q accelerate
!pip install -q evaluate
!pip install -q rouge-score
!pip install -q sentencepiece
!pip install -q protobuf
!pip install -q gradio

# Hugging Face Hub 로그인 (필요시)
from huggingface_hub import notebook_login
# notebook_login()  # 필요한 경우 주석 해제

In [ ]:
import torch
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training
)
from trl import SFTTrainer
import warnings
warnings.filterwarnings('ignore')

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 디바이스: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2. 데이터 로딩 및 전처리

In [ ]:
# Google Drive 연결 (데이터 파일이 Drive에 있는 경우)
from google.colab import drive
drive.mount('/content/drive')

# 또는 직접 파일 업로드
from google.colab import files
# uploaded = files.upload()  # 필요시 주석 해제

In [ ]:
# 데이터셋 파일 경로 설정 (Google Drive 경로로 수정)
data_files = {
    '구어체_대화체': '/content/drive/MyDrive/구어체_대화체_16878_sample_난독화결과.csv',
    '뉴스문어체': '/content/drive/MyDrive/뉴스문어체_281932_sample_난독화결과.csv',
    '문화문어체': '/content/drive/MyDrive/문화문어체_25628_sample_난독화결과.csv',
    '전문분야문어체': '/content/drive/MyDrive/전문분야 문어체_306542_sample_난독화결과.csv',
    '조례문어체': '/content/drive/MyDrive/조례문어체_36339_sample_난독화결과.csv',
    '지자체웹사이트문어체': '/content/drive/MyDrive/지자체웹사이트 문어체_28705_sample_난독화결과.csv'
}

# 모든 데이터셋 로드 및 결합
all_data = []
for name, file_path in data_files.items():
    try:
        df = pd.read_csv(file_path)
        df['category'] = name
        all_data.append(df)
        print(f"{name}: {len(df)} 샘플 로드됨")
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        continue

# 데이터 결합
combined_df = pd.concat(all_data, ignore_index=True)
print(f"\n전체 데이터셋 크기: {len(combined_df)}")
print(f"카테고리별 분포:")
print(combined_df['category'].value_counts())

In [ ]:
# 데이터 전처리 함수
def create_instruction_dataset(df, sample_size=None):
    """난독화 해제 작업을 위한 instruction 형태 데이터셋 생성"""

    if sample_size:
        df = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

    instructions = []
    for _, row in df.iterrows():
        # Instruction 형태로 변환
        instruction = {
            'input': f"다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.\n\n난독화된 텍스트: {row['obfuscated']}",
            'output': row['original'],
            'category': row['category']
        }
        instructions.append(instruction)

    return pd.DataFrame(instructions)

# Instruction 데이터셋 생성 (메모리 절약을 위해 샘플링)
sample_size = 10000  # 필요에 따라 조정
instruction_df = create_instruction_dataset(combined_df, sample_size)

print(f"Instruction 데이터셋 크기: {len(instruction_df)}")
print("\n첫 번째 예시:")
print(f"Input: {instruction_df.iloc[0]['input']}")
print(f"Output: {instruction_df.iloc[0]['output']}")

In [ ]:
# 훈련/검증 데이터 분할
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    instruction_df,
    test_size=0.1,
    random_state=42,
    stratify=instruction_df['category']
)

print(f"훈련 데이터: {len(train_df)}")
print(f"검증 데이터: {len(val_df)}")

# Hugging Face Dataset 형태로 변환
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

print("\n데이터셋 구조:")
print(dataset)

## 3. 모델 및 토크나이저 설정

In [ ]:
# 모델 설정
model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B"

# 양자화 설정 (메모리 절약)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(model_name)

# LoRA를 위한 모델 준비
model = prepare_model_for_kbit_training(model)

print(f"모델 로드 완료: {model_name}")
print(f"토크나이저 어휘 크기: {len(tokenizer)}")

In [ ]:
# LoRA 설정
lora_config = LoraConfig(
    r=16,  # rank
    lora_alpha=32,  # scaling parameter
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# LoRA 모델 생성
model = get_peft_model(model, lora_config)

# 훈련 가능한 파라미터 확인
model.print_trainable_parameters()

## 4. 데이터 포맷팅

In [ ]:
# 데이터 포맷팅 함수
def format_instruction(example):
    """Instruction 포맷으로 데이터 변환"""
    prompt = f"### 지시사항:\n{example['input']}\n\n### 응답:\n{example['output']}<|endoftext|>"
    return {"text": prompt}

# 데이터셋에 포맷 적용
formatted_dataset = dataset.map(format_instruction, remove_columns=dataset['train'].column_names)

print("포맷된 데이터 예시:")
print(formatted_dataset['train'][0]['text'][:500] + "...")

## 5. 모델 훈련

In [ ]:
# 훈련 설정
training_args = TrainingArguments(
    output_dir="./hyperclova-deobfuscation-lora",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    learning_rate=2e-4,
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=True,
    dataloader_pin_memory=False,
    remove_unused_columns=False,
    report_to="none"
)

print("훈련 설정 완료")

In [ ]:
# SFT Trainer 설정
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset['train'],
    eval_dataset=formatted_dataset['validation'],
    args=training_args
)

print("SFT Trainer 설정 완료")

In [ ]:
# 훈련 시작
print("훈련을 시작합니다...")
trainer.train()

# 모델 저장
trainer.save_model()
tokenizer.save_pretrained("./hyperclova-deobfuscation-lora")

print("훈련 완료 및 모델 저장됨")

## 6. 모델 평가 및 테스트

In [ ]:
# 추론 함수
def generate_deobfuscated_text(obfuscated_text, max_length=256):
    """난독화된 텍스트를 입력받아 원본 텍스트 생성"""
    prompt = f"### 지시사항:\n다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.\n\n난독화된 텍스트: {obfuscated_text}\n\n### 응답:\n"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 응답 부분만 추출
    if "### 응답:" in response:
        response = response.split("### 응답:")[1].strip()

    return response

print("추론 함수 정의 완료")

In [ ]:
# 테스트 샘플 평가
test_samples = val_df.sample(n=10, random_state=42)

print("=== 모델 테스트 결과 ===")
for idx, row in test_samples.iterrows():
    obfuscated = row['input'].split("난독화된 텍스트: ")[1]
    original = row['output']
    predicted = generate_deobfuscated_text(obfuscated)

    print(f"\n[{idx}] 카테고리: {row['category']}")
    print(f"난독화: {obfuscated}")
    print(f"원본: {original}")
    print(f"예측: {predicted}")
    print("-" * 80)

In [ ]:
# 정량적 평가 (BLEU, ROUGE 스코어)
from evaluate import load

# 메트릭 로드
bleu = load("bleu")
rouge = load("rouge")

def calculate_metrics(predictions, references):
    """BLEU, ROUGE 스코어 계산"""
    # BLEU 계산
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

    # ROUGE 계산
    rouge_score = rouge.compute(predictions=predictions, references=references)

    return {
        'bleu': bleu_score['bleu'],
        'rouge1': rouge_score['rouge1'],
        'rouge2': rouge_score['rouge2'],
        'rougeL': rouge_score['rougeL']
    }

# 100개 샘플로 평가
eval_samples = val_df.sample(n=100, random_state=42)
predictions = []
references = []

print("평가 중...")
for idx, row in eval_samples.iterrows():
    obfuscated = row['input'].split("난독화된 텍스트: ")[1]
    original = row['output']
    predicted = generate_deobfuscated_text(obfuscated)

    predictions.append(predicted)
    references.append(original)

    if len(predictions) % 20 == 0:
        print(f"{len(predictions)}/100 완료")

# 메트릭 계산
metrics = calculate_metrics(predictions, references)

print("\n=== 평가 결과 ===")
print(f"BLEU Score: {metrics['bleu']:.4f}")
print(f"ROUGE-1: {metrics['rouge1']:.4f}")
print(f"ROUGE-2: {metrics['rouge2']:.4f}")
print(f"ROUGE-L: {metrics['rougeL']:.4f}")

## 7. 인터랙티브 데모

In [ ]:
# Gradio 인터페이스
import gradio as gr

def deobfuscate_interface(obfuscated_text):
    """Gradio 인터페이스용 함수"""
    if not obfuscated_text.strip():
        return "난독화된 텍스트를 입력해주세요."

    try:
        result = generate_deobfuscated_text(obfuscated_text)
        return result
    except Exception as e:
        return f"오류가 발생했습니다: {str(e)}"

# Gradio 인터페이스 생성
demo = gr.Interface(
    fn=deobfuscate_interface,
    inputs=gr.Textbox(
        label="난독화된 한국어 텍스트",
        placeholder="난독화된 텍스트를 입력하세요...",
        lines=3
    ),
    outputs=gr.Textbox(
        label="복원된 원본 텍스트",
        lines=3
    ),
    title="HyperCLOVAX 한국어 텍스트 비난독화",
    description="난독화된 한국어 텍스트를 원본 텍스트로 복원합니다.",
    examples=[
        ["안녀하쎼요, 반갑쏘니댜!"],
        ["오늬 날씨갸 맆이 좆네욘."],
        ["한큿어 쳬연어 처륄예 댕햔 연귝을 해보갰습닏댜."]
    ]
)

# 데모 실행
demo.launch(share=True)

## 8. 모델 저장 및 업로드

In [ ]:
# Hugging Face Hub에 모델 업로드 (선택사항)
# 먼저 Hugging Face에 로그인해야 합니다

from huggingface_hub import notebook_login
# notebook_login()

# 모델 업로드
# model.push_to_hub("your-username/hyperclova-korean-deobfuscation")
# tokenizer.push_to_hub("your-username/hyperclova-korean-deobfuscation")

print("모델 저장 완료")
print("로컬 경로: ./hyperclova-deobfuscation-lora")

In [ ]:
# 로컬에 저장된 모델을 zip 파일로 압축하여 다운로드
!zip -r hyperclova_deobfuscation_model.zip ./hyperclova-deobfuscation-lora

# Google Colab에서 파일 다운로드
from google.colab import files
files.download('hyperclova_deobfuscation_model.zip')

print("모델 파일 다운로드 준비 완료")

## 결론

이 노트북에서는 다음을 수행했습니다:

1. **데이터 준비**: 6가지 유형의 한국어 난독화 데이터셋을 로드하고 전처리
2. **모델 설정**: HyperCLOVAX-SEED-Text-Instruct-0.5B 모델에 LoRA를 적용
3. **Fine-tuning**: 효율적인 parameter-efficient fine-tuning 수행
4. **평가**: BLEU, ROUGE 스코어를 통한 정량적 평가
5. **데모**: Gradio를 활용한 인터랙티브 인터페이스 제공

### 다음 단계:
- 더 많은 데이터로 추가 훈련
- 하이퍼파라미터 튜닝
- 다양한 평가 메트릭 적용
- 실제 서비스 배포 고려

### 모델 사용법:
```python
# 저장된 모델 로드
from peft import PeftModel
base_model = AutoModelForCausalLM.from_pretrained("naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B")
model = PeftModel.from_pretrained(base_model, "./hyperclova-deobfuscation-lora")
tokenizer = AutoTokenizer.from_pretrained("./hyperclova-deobfuscation-lora")
```